In [2]:
# from DeepPurpose import utils, dataset
# from DeepPurpose import DTI as models
# import warnings
# warnings.filterwarnings("ignore")

In [3]:
# %pip install git+https://github.com/bp-kelley/descriptastorus

In [4]:
# %pip install rdkit-pypi pandas matplotlib numpy nltk lifelines prettytable scikit-learn tensorflow torch

In [5]:
import os
import utils
import DTI as models
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import sys
import molecule_vae
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import Draw
import zinc_grammar
import nltk
from functools import reduce
import numpy as np  
import pandas as pd
import json

2024-04-18 11:23:37.188294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 11:23:40.295506: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-18 11:23:41.784154: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [6]:
drug_encoding, target_encoding = 'CNN', 'CNN'
#drug_encoding, target_encoding = 'Morgan', 'Conjoint_triad'

In [7]:
df_bindingdb = pd.read_csv('../data/BindingDB_IC50_updated.csv')

selection = ['cmap_name', 'std_smiles', 'Target Name', 'BindingDB Target Chain Sequence', 'IC50 (nM)']
df_bindingdb_selection = df_bindingdb[selection]
df_bindingdb_selection['pIC50'] = df_bindingdb_selection['IC50 (nM)'].apply(lambda x: utils.convert_y_unit(x, 'nM', 'p'))
df_bindingdb_selection

/tmp/ipykernel_768887/44699891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bindingdb_selection['pIC50'] = df_bindingdb_selection['IC50 (nM)'].apply(lambda x: utils.convert_y_unit(x, 'nM', 'p'))


,cmap_name,std_smiles,Target Name,BindingDB Target Chain Sequence,IC50 (nM),pIC50
0,indinavir,CC(C)(C)NC(=O)[C@@H]1CN(Cc2cccnc2)CCN1C[C@@H](...,Dimer of Gag-Pol polyprotein [501-599],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,0.36,9.443697
1,saquinavir,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Dimer of Gag-Pol polyprotein [489-587],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.40,8.075721
2,amprenavir,CC(C)CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)O[C@H]...,Dimer of Gag-Pol polyprotein [489-587],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2.60,8.585027
3,saquinavir,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Dimer of Gag-Pol polyprotein [489-587],PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,11.20,7.950782
4,staurosporine,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,Protein kinase C beta type,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,9.00,8.045757
...,...,...,...,...,...,...
18810,scriptaid,O=C(CCCCCN1C(=O)c2cccc3cccc(c23)C1=O)NO,Histone deacetylase 9,MHSMISSVDVKSEVPVGLEPISPLDLRTDLRMMMPVVDPVVREKQL...,700.00,6.154902
18811,entinostat,Nc1ccccc1NC(=O)c1ccc(CNC(=O)OCc2cccnc2)cc1,Histone deacetylase 9,MHSMISSVDVKSEVPVGLEPISPLDLRTDLRMMMPVVDPVVREKQL...,100000.00,4.000000
18812,vorinostat,O=C(CCCCCCC(=O)Nc1ccccc1)NO,Polyamine deacetylase HDAC10,MGTALVYHEDMTATRLLWDDPECEIERPERLTAALDRLRQRGLEQR...,200.00,6.698970
18813,scriptaid,O=C(CCCCCN1C(=O)c2cccc3cccc(c23)C1=O)NO,Polyamine deacetylase HDAC10,MGTALVYHEDMTATRLLWDDPECEIERPERLTAALDRLRQRGLEQR...,300.00,6.522879


In [8]:
X_drugs = df_bindingdb_selection['std_smiles']
X_targets = df_bindingdb_selection['BindingDB Target Chain Sequence']
y = df_bindingdb_selection['pIC50']

In [9]:
train, val, test = utils.data_process(X_drugs, X_targets, y, 
                                drug_encoding, target_encoding, 
                                split_method='random',frac=[0.7,0.1,0.2],
                                random_seed = 1)
train.head(1)

Drug Target Interaction Prediction Mode...
in total: 18815 drug-target pairs
encoding drug...
unique drugs: 2264
encoding protein...
unique target sequence: 1963
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CC(C)(C)NC(=O)[C@@H]1CN(Cc2cccnc2)CCN1C[C@@H](...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,9.443697,"[C, C, (, C, ), (, C, ), N, C, (, =, O, ), [, ...","[P, Q, I, T, L, W, Q, R, P, L, V, T, I, K, I, ..."


In [10]:
# bad_smiles = ['O=c1c2ccccc2[se]n1-c1ccccc1',
#  'CCP(CC)(CC)=[Au]S[C@@H]1O[C@H](COC(C)=O)[C@@H](OC(C)=O)[C@H](OC(C)=O)[C@H]1OC(C)=O']

In [11]:
config = utils.generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [1024,1024,512], 
                         train_epoch = 5, 
                         LR = 0.001, 
                         batch_size = 128,
                         cnn_drug_filters = [32,64,96],
					     cnn_drug_kernels = [4,6,8],
                         cnn_target_filters = [32,64,96],
                         cnn_target_kernels = [4,8,12]
                        )

In [12]:
model = models.model_initialize(**config)
model


libgomp: Thread creation failed: Resource temporarily unavailable


: 

In [ ]:
# model.train(train, val, test)